# The Data Science Pipeline: Exploring Video Game Ratings From IGN (and their sales) Over the Past 10 Years

## Introduction

### Hello and welcome! 
This is a walkthough of the Data Science Pipeline, using Python as the language of choice. Before we begin, let's first talk about our dependencies:
>This tutorial was created on a system running macOS. 

>Depending on your operating system, you may already have version of Python installed.

>However, any operating system should work. For more information, visit https://www.python.org/downloads/operating-systems/

### Jupyter & IPython
If you don't already know, this github page is actually a Jupyter (IPython) notebook that allows you to document both the input and output of code with boxes of text (known as "Markdown cells" - http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html#). These cells support comment styling augmented with a lightweight and easy-to-use markup langugae (https://daringfireball.net/projects/markdown/syntax).

Although not technically required, using an application that supports interactive literate programming makes the entire pipeline process far easier to document, understand, and benefit from; using one is highly recommended (especially for beginners and newcomers). 

> You may download Jupyter through Anaconda: http://jupyter.readthedocs.io/en/latest/install.html

### Python & Jupyter/IPython Version
I am using Python version 3.6.2 and Jupyter/IPython version 6.1.0. You may verify your versions via
> Help ---> About

Once you have your environment set up, it's time to get the data.

In [21]:
import pandas as pd
from datetime import datetime


df1 = pd.read_csv('/home/jovyan/notebooks/Final Project/dkanney.github.io/data/ign.csv')
df2 = pd.read_csv('/home/jovyan/notebooks/Final Project/dkanney.github.io/data/vgsales.csv')


# Restrict to games made from 2006-2016 inclusive
df1 = df1[df1['release_year'] > 2005]
# display('df1: ', df1[df1['title'] == 'Sonic the Hedgehog'], 'len: ', len(df1), 'df2: ', df2.head(10), 'len: ', len(df2))


# Join the two dataframes based on video game title/Name, getting rid of anything that we don't have release date.
raw_data = pd.merge(df1, df2, left_on='title', right_on='Name', how='inner')


# Using release year, release month, and release day columns to create a new column for release date as a datetime object
for i, row in raw_data.iterrows():
    raw_data.loc[i, 'release_date'] = pd.to_datetime(str(row['release_year']) + '-' + str(row['release_month']) + '-' + str(row['release_day']))

    
# Getting rid of unsued/duplicate columns
del raw_data['release_month']
del raw_data['release_year']
del raw_data['release_day']
del raw_data['Unnamed: 0']
del raw_data['genre']
del raw_data['Year']
del raw_data['Name']
del raw_data['url']


display('raw_data: ', raw_data.ix[485:500], 'len: ', len(raw_data))

'raw_data: '

,score_phrase,title,platform,score,editors_choice,Rank,Platform,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,release_date
485,Good,Mega Man X Collection,PlayStation 2,7.0,N,6779,PS2,Misc,Unknown,0.12,0.09,0.00,0.03,0.24,2006-01-05
486,Good,Mega Man X Collection,PlayStation 2,7.0,N,9641,GC,Misc,Capcom,0.10,0.03,0.00,0.00,0.13,2006-01-05
487,Good,Mega Man X Collection,GameCube,7.0,N,6779,PS2,Misc,Unknown,0.12,0.09,0.00,0.03,0.24,2006-01-05
488,Good,Mega Man X Collection,GameCube,7.0,N,9641,GC,Misc,Capcom,0.10,0.03,0.00,0.00,0.13,2006-01-05
489,Great,True Swing Golf,Nintendo DS,8.0,N,8496,DS,Sports,Nintendo,0.11,0.00,0.04,0.01,0.16,2006-01-20
490,Mediocre,Ape Escape Academy,PlayStation Portable,5.5,N,5463,PSP,Misc,Sony Computer Entertainment,0.13,0.15,0.00,0.05,0.33,2006-01-19
491,Bad,Monster Trucks DS,Nintendo DS,4.0,N,11598,DS,Racing,Majesco Entertainment,0.07,0.00,0.00,0.01,0.08,2006-01-17
492,Great,MVP 06 NCAA Baseball,PlayStation 2,8.5,N,2757,PS2,Sports,Electronic Arts,0.62,0.02,0.00,0.10,0.75,2006-01-13
493,Great,MVP 06 NCAA Baseball,PlayStation 2,8.5,N,5406,XB,Sports,Electronic Arts,0.25,0.07,0.00,0.01,0.34,2006-01-13
494,Great,MVP 06 NCAA Baseball,Xbox,8.7,N,2757,PS2,Sports,Electronic Arts,0.62,0.02,0.00,0.10,0.75,2006-01-13


'len: '

13646

## Missing Data

Let's create a function in order to assist us with finding missing (nan) values in our dataframe:

In [22]:
# Check for NaNs in the dataframe, printing 'True' followed by the number of records with 'nan'. 
# If the user specifies True for 'printrows', all entries containing 'nan' will be printed as well.
def check_for_nan(raw_data, printrows=False):
    print(raw_data.isnull().values.any())
    print(raw_data.isnull().sum().sum())

    if (printrows):
        # Find rows containing NaNs
        nan_rows = raw_data[raw_data.isnull().T.any().T]

        for i, row in nan_rows.iterrows():
            display(row)

Running the next cell returns the remaining records with 'NaN' entries

In [24]:
check_for_nan(raw_data, printrows=True)

True
7


score_phrase                      Bad
title              Sonic the Hedgehog
platform                     Xbox 360
score                             4.8
editors_choice                      N
Rank                             4147
Platform                          PS3
Genre                        Platform
Publisher                         NaN
NA_Sales                            0
EU_Sales                         0.48
JP_Sales                            0
Other_Sales                         0
Global_Sales                     0.48
release_date      2006-11-30 00:00:00
Name: 2455, dtype: object

score_phrase                      Bad
title              Sonic the Hedgehog
platform                PlayStation 3
score                             4.2
editors_choice                      N
Rank                             4147
Platform                          PS3
Genre                        Platform
Publisher                         NaN
NA_Sales                            0
EU_Sales                         0.48
JP_Sales                            0
Other_Sales                         0
Global_Sales                     0.48
release_date      2007-02-07 00:00:00
Name: 2459, dtype: object

score_phrase                    Great
title                    Mario Tennis
platform                          Wii
score                             8.5
editors_choice                      Y
Rank                            15264
Platform                          3DS
Genre                          Sports
Publisher                         NaN
NA_Sales                            0
EU_Sales                            0
JP_Sales                         0.02
Other_Sales                         0
Global_Sales                     0.02
release_date      2010-06-30 00:00:00
Name: 9305, dtype: object

score_phrase                     Good
title                  World of Tanks
platform                           PC
score                             7.5
editors_choice                      N
Rank                            13964
Platform                         X360
Genre                         Shooter
Publisher                         NaN
NA_Sales                            0
EU_Sales                         0.03
JP_Sales                            0
Other_Sales                         0
Global_Sales                     0.04
release_date      2011-07-21 00:00:00
Name: 10880, dtype: object

score_phrase                    Great
title                  World of Tanks
platform                     Xbox 360
score                               8
editors_choice                      N
Rank                            13964
Platform                         X360
Genre                         Shooter
Publisher                         NaN
NA_Sales                            0
EU_Sales                         0.03
JP_Sales                            0
Other_Sales                         0
Global_Sales                     0.04
release_date      2014-03-26 00:00:00
Name: 10882, dtype: object

score_phrase                 Mediocre
title                    Stronghold 3
platform                           PC
score                             5.5
editors_choice                      N
Rank                             8505
Platform                           PC
Genre                        Strategy
Publisher                         NaN
NA_Sales                         0.06
EU_Sales                          0.1
JP_Sales                            0
Other_Sales                         0
Global_Sales                     0.16
release_date      2011-11-04 00:00:00
Name: 11247, dtype: object

score_phrase                              Amazing
title             Homeworld Remastered Collection
platform                                       PC
score                                           9
editors_choice                                  Y
Rank                                        16194
Platform                                       PC
Genre                                    Strategy
Publisher                                     NaN
NA_Sales                                        0
EU_Sales                                     0.01
JP_Sales                                        0
Other_Sales                                     0
Global_Sales                                 0.01
release_date                  2015-02-24 00:00:00
Name: 13201, dtype: object

We now see that many of these rows contain missing/'NaN' values under 'Publisher'.
    
### Missing Publisher names?
Let's begin by looking up the missing Publishers and filling in the missing data points (see specific entries above). Since there doesn't seem to be a clear pattern or reason for why this data is missing, we will assume for now that it is Missing Completely at Random (MCAR). Read more about the three types of missing data at 
    
    https://en.wikipedia.org/wiki/Missing_data#Techniques_of_dealing_with_missing_data

Due to the small amount of missing Publisher names, we can use this moment to learn a bit more about our data set while imputing the missing data.

In [17]:
# Publisher --> 'Sega'
# (SOURCE: https://en.wikipedia.org/wiki/Sonic_the_Hedgehog_(2006_video_game))
raw_data.loc[2455, 'Publisher'] = 'Sega'
raw_data.loc[2459, 'Publisher'] = 'Sega'


# Publisher --> 'Nintendo'
# (SOURCE: https://en.wikipedia.org/wiki/Mario_Tennis)
raw_data.loc[9305, 'Publisher'] = 'Nintendo'


# Publisher --> 'Wargaming'
# (SOURCE: https://en.wikipedia.org/wiki/World_of_Tanks)
raw_data.loc[10880, 'Publisher'] = 'Wargaming' # Initial release
raw_data.loc[10882, 'Publisher'] = 'Wargaming' # Console edition


# Publisher --> '7Sixty'
# (SOURCE: https://en.wikipedia.org/wiki/Stronghold_3)
raw_data.loc[11247, 'Publisher'] = '7Sixty'


# Publisher --> 'Gearbox'
# (SOURCE: http://store.steampowered.com/app/244160/Homeworld_Remastered_Collection/)
raw_data.loc[13201, 'Publisher'] = 'Gearbox'

> Just how many Publishers are there?

In [55]:
tmp = raw_data.Publisher.unique()
tmp.sort()
print(tmp)

['Sony Computer Entertainment' 'Electronic Arts' 'NCSoft'
 'Namco Bandai Games' 'Nippon Ichi Software' 'Activision' 'Capcom'
 'Take-Two Interactive' 'Tecmo Koei' 'Ubisoft Annecy' 'Rondomedia' 'THQ'
 'Nordic Games' 'Microsoft Game Studios' 'Acclaim Entertainment' 'Sega'
 'GungHo' 'Square Enix' 'Konami Digital Entertainment' 'Rising Star Games'
 'Agatsuma Entertainment' 'Daedalic' 'Codemasters' 'Bethesda Softworks'
 'Ubisoft' 'Focus Home Interactive' 'Nintendo' 'FuRyu'
 'Warner Bros. Interactive Entertainment' 'PQube' 'Unknown'
 'Majesco Entertainment' '505 Games' 'D3Publisher' 'Midway Games' 'Atari'
 'Oxygen Interactive' 'Crave Entertainment' 'Zoo Digital Publishing'
 'Vivendi Games' 'Ignition Entertainment' 'LucasArts' 'Ocean'
 'SouthPeak Games' 'Valcon Games' 'Tomy Corporation'
 'Midas Interactive Entertainment' 'Spike' 'Mad Catz' 'Empire Interactive'
 'Eidos Interactive' 'Sony Online Entertainment' 'Deep Silver'
 'Myelin Media' 'Success' 'Global Star' 'Genki' 'Activision Value'
 'JoW

There appears to be Publishers with similar strings (i.e. 'Valve' and 'Valve Software'). We can group these together to simplify analysis -- within reason.

In [53]:
for i in raw_data[raw_data['Publisher'] == 'Valve Software'].index:
    raw_data.loc[i, 'Publisher'] = 'Valve'

    You have reached the end of the Tutorial.

In [1]:
# import math
# import sklearn
# import numpy as np
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn import linear_model, datasets
# import statsmodels.api as sm_api
# import statsmodels.formula.api as sm
# from statsmodels.api import add_constant
# from sklearn.preprocessing import PolynomialFeatures
# from ggplot import aes, geom_violin, geom_point, geom_boxplot, geom_line, labs, ggplot


# df = pd.read_csv("https://raw.githubusercontent.com/jennybc/gapminder/master/data-raw/08_gap-every-five-years.tsv", sep='\t')
# df